<a href="https://colab.research.google.com/github/bhuiyanmobasshir94/Made-With-ML/blob/main/machine-learning/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
SEED = 1234
NUM_SAMPLES = 50

In [3]:
# Set seed for reproducibility
np.random.seed(SEED)

In [4]:
# Generate synthetic data
def generate_data(num_samples):
    """Generate dummy data for linear regression."""
    X = np.array(range(num_samples))
    print(f"X: {X}")
    random_noise = np.random.uniform(-10, 20, size=num_samples)
    print(f"random_noise: {random_noise}")
    y = 3.5*X + random_noise # add some noise
    return X, y

In [9]:
# Generate random (linear) data
X, y = generate_data(num_samples=NUM_SAMPLES)
print(f"X: {X}")
print(f"y: {y}")
data = np.vstack([X, y]).T
print (data[:5])

X: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
random_noise: [ 1.30134989e+01  1.12434609e+01  1.39060155e+01  6.73282485e+00
  1.89750960e+01 -5.58529300e+00 -9.11058998e+00  7.81680478e+00
 -6.57802904e+00  1.85242955e+01 -2.28777567e-01 -4.19143930e+00
  3.73434947e+00  1.76120771e+01  1.63720748e+01 -2.42152735e+00
  4.40263786e-01 -4.52233805e+00  1.70538815e+01  1.11958449e+01
  1.17997538e+01  1.70026351e+01  1.33749140e+01  7.97464342e+00
 -1.26624265e+00 -5.45814207e+00  5.52397745e-02  9.72655331e+00
 -7.79972369e+00 -8.34980814e+00 -3.04155582e-01  7.71445413e+00
  1.56169570e+01 -1.38812725e+00 -4.80798320e+00 -5.97936382e+00
  1.98396149e+01 -4.61506392e+00 -4.73595309e-01  7.04874214e+00
 -9.71954276e+00  1.70194586e+01  1.93172429e+01  6.70684037e+00
 -7.45678470e+00 -9.92602813e-03  1.18528603e+01 -5.72693880e+00
  6.57406818e+00 -1.80870221e+00]
X: [ 0  1  2  3

In [10]:
print(f"data shape: {data.shape}")
print(f"data size: {data.size}")

data shape: (50, 2)
data size: 100
